# Réponses aux questions du TP :

Equipe : Artus Bleton & Guilhem Dupuy

Répartition du travail : gloablement travail réalisé ensemble et en échangeant, Deep Network code par Artus, CNN code par Guilhem

## 5 Part 1 : Perceptron
- Data train = taille images * nb images = 28 * 28 * nb images
- Label train = taille finale du vecteur * nb images  = 10 * nb images
- Logique similaire pour données de test
- Taille entrée = taille de w = data_train nb de colonnes * label_train*nombre de colonnes = 784 * 10
- Taille sortie = taille de b = 1 * label_train nb de colonnes = 1 * 10
- Taille de x = taille images * 5
- Taille de y = taille de 5 résultats = 5 * 10
- Taille de t = comme y
- Taille of grad = 5 * 10. Intuitivement celui-ci je ne l'ai pas, commment ce nb va-t-il augmenter quand nous allons appliquer la chain rule ?
- 

### Ce que donne le code : 
Taille entrée = 784, taille sortie = 10
 size of data_train : torch.Size([63000, 784])
 size of label_train : torch.Size([63000, 10])
 size of data_test : torch.Size([7000, 784])
 size of label_test : torch.Size([7000, 10])
 size of w : torch.Size([784, 10])
 size of b : torch.Size([1, 10])
 size of x : torch.Size([5, 784])
 size of y : torch.Size([5, 10])
 size of grad : torch.Size([5, 10])

In [2]:
# perceptron_pytorch

import gzip, numpy, torch
    
batch_size = 5 # nombre de données lues à chaque fois
nb_epochs = 10 # nombre de fois que la base de données sera lue
eta = 0.00001 # taux d'apprentissage

# on lit les données
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('mnist_TP1.pkl.gz'))

# on initialise le modèle et ses poids
w = torch.empty((data_train.shape[1],label_train.shape[1]),dtype=torch.float)
print(f"Taille entrée = {data_train.shape[1]}, taille sortie = {label_train.shape[1]}")
b = torch.empty((1,label_train.shape[1]),dtype=torch.float)
torch.nn.init.uniform_(w,-0.001,0.001)
torch.nn.init.uniform_(b,-0.001,0.001)

#Affichage des tailles
print(f" size of data_train : {data_train.shape}")
print(f" size of label_train : {label_train.shape}")
print(f" size of data_test : {data_test.shape}")
print(f" size of label_test : {label_test.shape}")
print(f" size of w : {w.shape}")
print(f" size of b : {b.shape}")

nb_data_train = data_train.shape[0]
nb_data_test = data_test.shape[0]
indices = numpy.arange(nb_data_train)

for n in range(nb_epochs):
    # on mélange les (indices des) données
    numpy.random.shuffle(indices)
    # on lit toutes les (indices des) données d'apprentissage
    for i in range(0,nb_data_train,batch_size):
        # on récupère les entrées
        x = data_train[indices[i:i+batch_size]]
        # on calcule la sortie du modèle
        y = torch.mm(x,w)+b
        # on regarde les vrais labels
        t = label_train[indices[i:i+batch_size]]
        # on met à jour les poids
        grad = (t-y)      #La fonction d'activation est donc linéaire je crois, avec une loss fonction de type somme des carrés ?
        w += eta * torch.mm(x.T,grad)
        b += eta * grad.sum(axis=0)

        if i == 0 and n == 1 : 
            print(f" size of x : {x.shape}")
            print(f" size of y : {y.shape}")
            print(f" size of grad : {grad.shape}")

    # test du modèle (on évalue la progression pendant l'apprentissage)
    acc = 0.
    # on lit toutes les donnéees de test
    for i in range(nb_data_test):
        # on récupère l'entrée
        x = data_test[i:i+1]
        # on calcule la sortie du modèle
        y = torch.mm(x,w)+b
        # on regarde le vrai label
        t = label_test[i:i+1]
        # on regarde si la sortie est correcte
        acc += torch.argmax(y,1) == torch.argmax(t,1)
    # on affiche le pourcentage de bonnes réponses
    print(acc/nb_data_test)


Taille entrée = 784, taille sortie = 10
 size of data_train : torch.Size([63000, 784])
 size of label_train : torch.Size([63000, 10])
 size of data_test : torch.Size([7000, 784])
 size of label_test : torch.Size([7000, 10])
 size of w : torch.Size([784, 10])
 size of b : torch.Size([1, 10])
tensor([0.8084])
 size of x : torch.Size([5, 784])
 size of y : torch.Size([5, 10])
 size of grad : torch.Size([5, 10])
tensor([0.8256])
tensor([0.8351])
tensor([0.8384])
tensor([0.8437])
tensor([0.8476])
tensor([0.8509])
tensor([0.8523])
tensor([0.8531])
tensor([0.8526])


# 6 Part 2 : Shallow Network - Grid Search

In [3]:
# 6 Part 2 : 

#Class MyModel

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

class MyModel(nn.Module):
    def __init__(self, nbfeatures=784, k_intermediate_neurons=100):
        self.k_intermediate_neurons = k_intermediate_neurons
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(nbfeatures, k_intermediate_neurons)
        self.fc2 = nn.Linear(k_intermediate_neurons,10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# on lit les données
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('mnist_TP1.pkl.gz'))

#Hyperparamètres
k_intermediate_neurons = [1000,500,100,20]
batch_size = [16,32,64,128] # nombre de données lues à chaque fois
eta = [0.01,0.0075,0.005,0.0025] # taux d'apprentissage

# on crée les lecteurs de données
net = NeuralNetClassifier(
    MyModel,
    max_epochs=20,
    lr=0.001,
    batch_size=64,
    optimizer=torch.optim.SGD,
    criterion=nn.CrossEntropyLoss,
    verbose=1
)

X_trf = data_train.numpy()
y_trf = torch.argmax(label_train, dim=1).numpy()

#X_trf = train_dataset
#y_trf = test_loader.reshape(-1, 1)
print(X_trf.shape,y_trf.shape)

params = {
    'lr': [0.01,0.0001],
    'batch_size': [16,128],
    'module__k_intermediate_neurons': [1000,20]
}

gs = GridSearchCV(net, params, refit=False, scoring='accuracy', verbose=1, cv=2)

gs.fit(X_trf, y_trf)

(63000, 784) (63000,)
Fitting 2 folds for each of 8 candidates, totalling 16 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.8606       0.8695        0.4523  2.0200
      2        0.3825       0.8924        0.3623  1.8158
      3        0.3225       0.9025        0.3249  1.8368
      4        0.2886       0.9098        0.3000  1.8663
      5        0.2630       0.9154        0.2803  1.9904
      6        0.2418       0.9210        0.2635  1.9226
      7        0.2236       0.9260        0.2489  1.8645
      8        0.2076       0.9289        0.2360  1.9185
      9        0.1934       0.9329        0.2246  2.1750
     10        0.1807       0.9363        0.2144  2.2650
     11        0.1693       0.9394        0.2053  2.5278
     12        0.1590       0.9410        0.1971  2.3767
     13        0.1495       0.9432        0.1895  2.5524
     14        0.1410       0.9449        0.1827  2.5751
     1

GridSearchCV(cv=2,
             estimator=NeuralNetClassifier(_params_to_validate=set(), batch_size=64, callbacks=None, compile=False, criterion=<class 'torch.nn.modules.loss.CrossEntropyLoss'>, dataset=<class 'skorch.dataset.Dataset'>, device='cpu', iterator_train=<class 'torch.utils.data.dataloader.DataLoader'>, iterator_valid=<class 'torch.utils.data.dataloader.DataLoader'>, lr=0.001, max_epochs=20, module=<class '__main__.MyModel'>, optimizer=<class 'torch.optim.sgd.SGD'>, predict_nonlinearity='auto', torch_load_kwargs=None, use_caching='auto', verbose=1, warm_start=False),
             param_grid={'batch_size': [16, 128], 'lr': [0.01, 0.0001],
                         'module__k_intermediate_neurons': [1000, 20]},
             refit=False, scoring='accuracy', verbose=1)

# 6 bis : GridSearch

In [4]:
# on lit les données
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('mnist_TP1.pkl.gz'))

#Hyperparamètres
k_intermediate_neurons = 500
batch_size = 64 # nombre de données lues à chaque fois
nb_epochs = 30 # nombre de fois que la base de données sera lue
eta = 0.005 # taux d'apprentissage

# on crée les lecteurs de données

#on extrait des données de validation de data_train, pour avoir 7000/7000/56000 soit 10/10/80%
data_validation = data_train[:7000]
data_train = data_train[7000:]
label_validation = label_train[:7000]
label_train = label_train[7000:]

train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
validation_dataset = torch.utils.data.TensorDataset(data_validation,label_validation)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=False)

print(f"size comparison train / test / validation : {data_train.shape} / {data_test.shape} / {data_validation.shape}")

# on initialise le modèle et ses poids
model = MyModel(data_train.shape[1], k_intermediate_neurons)

# on initiliase l'optimiseur
loss_func = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.SGD(model.parameters(), lr=eta)

for n in range(nb_epochs):
    # on lit toutes les données d'apprentissage
    for x,t in train_loader:
        model.train()
        optim.zero_grad()
        # on calcule la sortie du modèle
        y = model(x)
        # on met à jour les poids
        loss = loss_func(t,y)
        loss.backward()
        optim.step()
        
    # test du modèle (on évalue la progression pendant l'apprentissage)
    acc = 0
    # on lit toutes les donnéees de test
    for x,t in test_loader:
        # on calcule la sortie du modèle
        y = model(x)
        # on regarde si la sortie est correcte
        acc += torch.argmax(y,1) == torch.argmax(t,1)
    # on affiche le pourcentage de bonnes réponses
    print(acc/data_test.shape[0])


acc = 0
for x,t in validation_loader :
    # on calcule la sortie du modèle
    y = model(x)
    # on regarde si la sortie est correcte
    acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(f"Avec données de validation : {acc/data_validation.shape[0]}")

# Comment nous avons optimisé les Hyper-paramètres : 
# Sans puissance de calcul, nous avons fait ça manuellement jusqu'à obtenir des résultats satisfaisants. 
#1 - Nous avons augmenté le learning rate jusqu'à voir l'overfitting apparaître en 10 epochs
#2 - Taille du batch : choix d'une taille donant de bonnes performances et de bons résultats
#3 - Taille de la couche intermédiare : essai de 5 valeurs, choix de la meilleure option

size comparison train / test / validation : torch.Size([56000, 784]) / torch.Size([7000, 784]) / torch.Size([7000, 784])
tensor([0.9537])
tensor([0.9650])
tensor([0.9707])
tensor([0.9737])
tensor([0.9746])
tensor([0.9731])
tensor([0.9777])
tensor([0.9769])
tensor([0.9764])
tensor([0.9791])
tensor([0.9789])
tensor([0.9811])
tensor([0.9806])
tensor([0.9801])
tensor([0.9790])
tensor([0.9800])
tensor([0.9797])
tensor([0.9799])
tensor([0.9799])
tensor([0.9797])
tensor([0.9780])
tensor([0.9810])
tensor([0.9813])
tensor([0.9807])
tensor([0.9804])
tensor([0.9799])
tensor([0.9799])
tensor([0.9813])
tensor([0.9809])
tensor([0.9801])
Avec données de validation : tensor([0.9770])


In [5]:
acc = 0
for x,t in validation_loader :
    # on calcule la sortie du modèle
    y = model(x)
    # on regarde si la sortie est correcte
    acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(acc/data_validation.shape[0])

tensor([0.9770])


# 7  Part 3 : Deep network

In [6]:
import gzip,numpy,torch
import torch.nn as nn


class ShallowNet(nn.Module):
    def __init__(self, size_train, size_label, a, b, c):
        super().__init__()
        self.layer1 = torch.nn.Linear(size_train, a)
        self.relu1 = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(a, b)
        self.relu2 = torch.nn.ReLU()
        self.layer3 = torch.nn.Linear(b, c)
        self.relu3 = torch.nn.ReLU()
        self.layer4 = torch.nn.Linear(c, size_label)
        self.sm = torch.nn.Softmax(dim=1)


    def forward(self, x):
        res = self.layer1(x)
        res = self.relu1(res)
        res = self.layer2(res)
        res = self.relu2(res)
        res = self.layer3(res)
        res = self.relu3(res)
        res = self.layer4(res)
        res = self.sm(res)
        return res


def train_perso(a, b, c) :

    #Hyperparamètres
    batch_size = 64 # nombre de données lues à chaque fois
    nb_epochs = 30 # nombre de fois que la base de données sera lue
    eta = 0.001 # taux d'apprentissage
    hidden_size = 500
    
    # on lit les données
    ((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('mnist_TP1.pkl.gz'))
    # on crée les lecteurs de données
    train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
    test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    
    model = ShallowNet(data_train.shape[1],label_train.shape[1], a, b, c)
    
    
    # on initiliase l'optimiseur
    loss_func = torch.nn.MSELoss(reduction='sum')
    optim = torch.optim.SGD(model.parameters(), lr=eta)

    
    acc = 0
    for n in range(nb_epochs):
        # on lit toutes les données d'apprentissage
        for x,t in train_loader:
            # on calcule la sortie du modèle
            y = model(x)
            # on met à jour les poids
            loss = loss_func(t,y)
            loss.backward()
            optim.step()
            optim.zero_grad()


        # test du modèle (on évalue la progression pendant l'apprentissage)
        acc_prec = acc
        acc = float(0)
        
        # on lit toutes les donnéees de test
        for x,t in test_loader:
            # on calcule la sortie du modèle
            y = model(x)
            # on regarde si la sortie est correcte
            acc += torch.argmax(y,1) == torch.argmax(t,1)

        print(f"epoch n : {n} acc : {acc/data_test.shape[0]}")

        #Arrêt prématuré si Overfitting
        if acc < acc_prec :
            
            print(acc_prec/data_test.shape[0])
            print(f"Arrêt au step {n} pour modèle {a} / {b} / {c}")
            break

    



train_perso(500,250,50)
train_perso(50,250,500)
train_perso(250,250,250)
train_perso(500,250,50)
train_perso(50,250,500)
train_perso(250,250,250)

# Conclusion : la taille des couches n'importe peu, car les données sont "simples" et qu'un petit réseau à 1 couche suffit. 
# Le jeu sur les hyperparamètres permettant de s'entraîner plus a plus d'impact

epoch n : 0 acc : tensor([0.8380])
epoch n : 1 acc : tensor([0.9141])
epoch n : 2 acc : tensor([0.9359])
epoch n : 3 acc : tensor([0.9509])
epoch n : 4 acc : tensor([0.9596])
epoch n : 5 acc : tensor([0.9650])
epoch n : 6 acc : tensor([0.9630])
tensor([0.9650])
Arrêt au step 6 pour modèle 500 / 250 / 50
epoch n : 0 acc : tensor([0.8860])
epoch n : 1 acc : tensor([0.9179])
epoch n : 2 acc : tensor([0.9281])
epoch n : 3 acc : tensor([0.9419])
epoch n : 4 acc : tensor([0.9474])
epoch n : 5 acc : tensor([0.9553])
epoch n : 6 acc : tensor([0.9596])
epoch n : 7 acc : tensor([0.9650])
epoch n : 8 acc : tensor([0.9661])
epoch n : 9 acc : tensor([0.9699])
epoch n : 10 acc : tensor([0.9690])
tensor([0.9699])
Arrêt au step 10 pour modèle 50 / 250 / 500
epoch n : 0 acc : tensor([0.8441])
epoch n : 1 acc : tensor([0.9129])
epoch n : 2 acc : tensor([0.9316])
epoch n : 3 acc : tensor([0.9463])
epoch n : 4 acc : tensor([0.9554])
epoch n : 5 acc : tensor([0.9586])
epoch n : 6 acc : tensor([0.9643])
epo

# 8 Part 4 : CNN

In [7]:
import gzip
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.relu1 = torch.nn.ReLU() #size : 24 * 24 * 6 
        self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu2 = torch.nn.ReLU() #size : 12 * 12 * 6
        self.layer3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu3 = torch.nn.ReLU() #size : 8 * 8 * 16
        self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu4 = torch.nn.ReLU() #size : 4 * 4 * 16

        # C5: Fully connected (Linear layer) (input: 16 * 4 * 4, output: 120)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)

        # F6: Fully connected (Linear layer) (input: 120, output: 84)
        self.fc2 = nn.Linear(120, 84)

        # Output layer (Fully connected, input: 84, output: 10)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        #On dé-flatten les données de base
        x = x.view(-1, 1, 28, 28)

        # Convolution + ReLU layers
        res = self.layer1(x)
        res = self.relu1(res)
        res = self.layer2(res)
        res = self.relu2(res)
        res = self.layer3(res)
        res = self.relu3(res)
        res = self.layer4(res)
        res = self.relu4(res)
        
        # Flatten the output of the convolution layers
        res = res.view(res.size(0), -1)  # Reshape to (batch_size, 16 * 4 * 4)
        
        # Fully connected layers
        res = self.fc1(res)
        res = F.relu(res)
        res = self.fc2(res)
        res = F.relu(res)
        res = self.fc3(res)  # Output layer
        return res

t# Hyperparameters
batch_size = 64  # number of data points per batch
nb_epochs = 30   # number of times to iterate over the dataset
eta = 0.005      # learning rate

# Load data
((data_train, label_train), (data_test, label_test)) = torch.load(gzip.open('mnist_TP1.pkl.gz'))

label_train = torch.argmax(label_train, dim=1)
label_test = torch.argmax(label_test, dim=1)

# Create datasets and dataloaders
train_dataset = torch.utils.data.TensorDataset(data_train, label_train)
test_dataset = torch.utils.data.TensorDataset(data_test, label_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Initialize model
model = CNN()

# Define loss function and optimizer
loss_func = torch.nn.CrossEntropyLoss()  # Change MSELoss to CrossEntropyLoss for classification
optim = torch.optim.SGD(model.parameters(), lr=eta)

# Training loop
for n in range(nb_epochs):
    model.train()  # Set model to training mode
    for x, t in train_loader:
        optim.zero_grad()  # Clear gradients
        y = model(x)       # Forward pass
        loss = loss_func(y, t)  # Compute loss
        loss.backward()    # Backward pass
        optim.step()       # Update weights

    # Test loop
    model.eval()  # Set model to evaluation mode
    acc = 0
    with torch.no_grad():  # No need to compute gradients during testing
        for x, t in test_loader:
            y = model(x)
            acc += (torch.argmax(y, 1) == t).item()
    print(f'Epoch {n+1}/{nb_epochs}, Accuracy: {acc / len(data_test)}')




Epoch 1/30, Accuracy: 0.3598571428571429
Epoch 2/30, Accuracy: 0.7684285714285715
Epoch 3/30, Accuracy: 0.8694285714285714
Epoch 4/30, Accuracy: 0.913
Epoch 5/30, Accuracy: 0.9215714285714286
Epoch 6/30, Accuracy: 0.944
Epoch 7/30, Accuracy: 0.9488571428571428
Epoch 8/30, Accuracy: 0.9601428571428572
Epoch 9/30, Accuracy: 0.9557142857142857
Epoch 10/30, Accuracy: 0.9575714285714285
Epoch 11/30, Accuracy: 0.9722857142857143
Epoch 12/30, Accuracy: 0.9734285714285714
Epoch 13/30, Accuracy: 0.9734285714285714
Epoch 14/30, Accuracy: 0.9764285714285714
Epoch 15/30, Accuracy: 0.9764285714285714
Epoch 16/30, Accuracy: 0.9748571428571429
Epoch 17/30, Accuracy: 0.979
Epoch 18/30, Accuracy: 0.977
Epoch 19/30, Accuracy: 0.9767142857142858
Epoch 20/30, Accuracy: 0.9782857142857143
Epoch 21/30, Accuracy: 0.9794285714285714
Epoch 22/30, Accuracy: 0.9801428571428571
Epoch 23/30, Accuracy: 0.9761428571428571
Epoch 24/30, Accuracy: 0.9802857142857143
Epoch 25/30, Accuracy: 0.9817142857142858
Epoch 26/30